In [204]:
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import json
from pandas.io.json import json_normalize

%matplotlib inline

init_notebook_mode(connected=True)

In [205]:
def str_to_list(x):
    return eval(x) if x and x != '#N/A' else []

list_cols = [
    'genres',
    'belongs_to_collection',
    'production_companies',
    'production_countries',
    'spoken_languages',
    'Keywords',
    'cast',
    'crew'
]

io_params = {
    'index_col': 'id',
    'converters': {col: str_to_list for col in list_cols}
}

# We can read the train and the test in one go using the concat function
data = pd.read_csv('../data/train.csv', **io_params)
# data = pd.concat(
#     objs=(
#         pd.read_csv('../data/train.csv', **io_params).assign(is_train=True),
#         pd.read_csv('../data/test.csv', **io_params).assign(is_train=False)
#     ),
#     sort=False
# )

In [207]:
# data.head(100)
data

,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,...,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
id,,,,,,,,,,,,,,,,,,,,,
1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,...,2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,...,8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
3,[],3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,...,10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
4,[],1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,/aTXRaPrWSinhcmCrcfJK17urp3F.jpg,...,3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
5,[],0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,/m22s7zvkVFDU9ir56PiiqIEWFdT.jpg,...,2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,[],"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970
6,[],8000000,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",NaN,tt0093743,en,Pinocchio and the Emperor of the Night,"Pinocchio and his friends, a glow worm and a m...",0.743274,/6IDqA1D2NBIVhzEEaMMRL28iBrq.jpg,...,8/6/87,83.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Pinocchio and the Emperor of the Night,[],"[{'cast_id': 6, 'character': 'Pinocchio (voice...","[{'credit_id': '52fe46f49251416c9106558b', 'de...",3261638
7,[],14000000,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",http://www.thepossessionmovie.com/,tt0431021,en,The Possession,A young girl buys an antique box at a yard sal...,7.286477,/4QjzFuaZmB4btGnLwAgdp23BzIU.jpg,...,8/30/12,92.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Fear The Demon That Doesn't Fear God,The Possession,[],"[{'cast_id': 23, 'character': 'Clyde', 'credit...","[{'credit_id': '52fe4981c3a368484e12ee29', 'de...",85446075
8,[],0,"[{'id': 99, 'name': 'Documentary'}]",NaN,tt0391024,en,Control Room,A chronicle which provides a rare window into ...,1.949044,/83BV8fGy5p9i8RwE9oj76tZ1Ikp.jpg,...,1/15/04,84.0,"[{'iso_639_1': 'ar', 'name': 'العربية'}, {'iso...",Released,Different channels. Different truths.,Control Room,"[{'id': 917, 'name': 'journalism'}, {'id': 163...","[{'cast_id': 2, 'character': 'Himself', 'credi...","[{'credit_id': '52

In [191]:
# build dictionary of id to title:
display(pd.Series(data['title'].values, index=data.index).to_dict())

{1: 'Hot Tub Time Machine 2',
 2: 'The Princess Diaries 2: Royal Engagement',
 3: 'Whiplash',
 4: 'Kahaani',
 5: 'Marine Boy',
 6: 'Pinocchio and the Emperor of the Night',
 7: 'The Possession',
 8: 'Control Room',
 9: 'Muppet Treasure Island',
 10: 'A Mighty Wind',
 11: 'Rocky',
 12: 'Revenge of the Nerds II: Nerds in Paradise',
 13: 'American Beauty',
 14: 'Be Cool',
 15: 'Minority Report',
 16: 'Skinning',
 17: 'Red Eye',
 18: 'The Invisible Woman',
 19: 'Chalet Girl',
 20: 'Transporter 2',
 21: 'Lost in Space',
 22: 'Friday the 13th Part III',
 23: 'V/H/S',
 24: 'InSight',
 25: 'Black Sheep',
 26: 'Dr. Giggles',
 27: 'The Spanish Prisoner',
 28: 'What If',
 29: 'The Transformers: The Movie',
 30: 'Changing Lanes',
 31: 'The Intouchables',
 32: 'Caché',
 33: 'The Last Witch Hunter',
 34: 'ABCs of Death 2',
 35: 'Rush Hour 3',
 36: 'Safe',
 37: 'About Adam',
 38: 'Outbreak',
 39: 'Final: The Rapture',
 40: 'Major Dundee',
 41: 'Cocaine Cowboys',
 42: 'Cool Hand Luke',
 43: 'The Next 

In [208]:
## Get rid of the columns that are not relevant:
## id, homepage, imdb_id, title, poster_path, 
drop_col_names = ['belongs_to_collection', 'budget', 'genres', 'original_language', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'runtime', 'spoken_languages', 'cast', 'crew', 'revenue']
clean = data[drop_col_names]


In [209]:
# https://ipython-books.github.io/71-exploring-a-dataset-with-pandas-and-matplotlib/

## EDA
clean.dtypes

# belongs_to_collection:

belongs_to_collection     object
budget                     int64
genres                    object
original_language         object
overview                  object
popularity               float64
production_companies      object
production_countries      object
release_date              object
runtime                  float64
spoken_languages          object
cast                      object
crew                      object
revenue                    int64
dtype: object

In [210]:
## budget:
data = [go.Scatter(x=clean['budget'], y=clean['revenue'], mode='markers')]
layout = go.Layout(
    xaxis=dict(
        range=[0, 1e8]
    ),
    yaxis=dict(
        range=[0, 1e9]
    )
)
iplot(go.Figure(data = data))

In [248]:
## genres: one hot encoding

def is_genre(column, genre_name):
    """
    input: 
    output: a vector of True/False
    """
    result = []
    for genre_list in column:
        belong_to_genre = False
        for item in genre_list:
            if item['name'] == genre_name:
                belong_to_genre = True
        result.append(belong_to_genre)
    return result
    
unique_genres = []
for genre_list in clean['genres']:
    for genre_dict in genre_list:
        if genre_dict['name'] not in unique_genres:
            unique_genres.append(genre_dict['name'])
            
print(unique_genres)

## One-Hot Encoding:
clean['Comedy'] = np.where(is_genre(clean['genres'], 'Comedy'), 1, 0)
clean['Drama'] = np.where(is_genre(clean['genres'], 'Drama'), 1, 0)
clean['Family'] = np.where(is_genre(clean['genres'], 'Family'), 1, 0)
clean['Romance'] = np.where(is_genre(clean['genres'], 'Romance'), 1, 0)
clean['Thriller'] = np.where(is_genre(clean['genres'], 'Thriller'), 1, 0)
clean['Action'] = np.where(is_genre(clean['genres'], 'Action'), 1, 0)
clean['Animation'] = np.where(is_genre(clean['genres'], 'Animation'), 1, 0)
clean['Adventure'] = np.where(is_genre(clean['genres'], 'Adventure'), 1, 0)
clean['Horror'] = np.where(is_genre(clean['genres'], 'Horror'), 1, 0)
clean['Documentary'] = np.where(is_genre(clean['genres'], 'Documentary'), 1, 0)
clean['Music'] = np.where(is_genre(clean['genres'], 'Music'), 1, 0)
clean['Crime'] = np.where(is_genre(clean['genres'], 'Crime'), 1, 0)
clean['Science Fiction'] = np.where(is_genre(clean['genres'], 'Science Fiction'), 1, 0)
clean['Mystery'] = np.where(is_genre(clean['genres'], 'Mystery'), 1, 0)
clean['Foreign'] = np.where(is_genre(clean['genres'], 'Foreign'), 1, 0)
clean['Fantasy'] = np.where(is_genre(clean['genres'], 'Fantasy'), 1, 0)
clean['War'] = np.where(is_genre(clean['genres'], 'War'), 1, 0)
clean['Western'] = np.where(is_genre(clean['genres'], 'Western'), 1, 0)
clean['History'] = np.where(is_genre(clean['genres'], 'History'), 1, 0)
clean['TV Movie'] = np.where(is_genre(clean['genres'], 'TV Movie'), 1, 0)



['Comedy', 'Drama', 'Family', 'Romance', 'Thriller', 'Action', 'Animation', 'Adventure', 'Horror', 'Documentary', 'Music', 'Crime', 'Science Fiction', 'Mystery', 'Foreign', 'Fantasy', 'War', 'Western', 'History', 'TV Movie']


/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stab

In [250]:
clean.head(50)

,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,production_countries,release_date,runtime,...,Music,Crime,Science Fiction,Mystery,Foreign,Fantasy,War,Western,History,TV Movie
id,,,,,,,,,,,,,,,,,,,,,
1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",en,"When Lou, who has become the ""father of the In...",6.575393,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.0,...,0,0,0,0,0,0,0,0,0,0
2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",en,Mia Thermopolis is now a college graduate and ...,8.248895,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.0,...,0,0,0,0,0,0,0,0,0,0
3,[],3300000,"[{'id': 18, 'name': 'Drama'}]",en,"Under the direction of a ruthless instructor, ...",64.299990,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,105.0,...,0,0,0,0,0,0,0,0,0,0
4,[],1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",hi,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,[],"[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,122.0,...,0,0,0,0,0,0,0,0,0,0
5,[],0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",ko,Marine Boy is the story of a former national s...,1.148070,[],"[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2/5/09,118.0,...,0,0,0,0,0,0,0,0,0,0
6,[],8000000,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",en,"Pinocchio and his friends, a glow worm and a m...",0.743274,[],[],8/6/87,83.0,...,0,0,0,0,0,0,0,0,0,0
7,[],14000000,"[{'id': 27, 'name': 'Horror'}, {'id': 53, 'nam...",en,A young girl buys an antique box at a yard sal...,7.286477,"[{'name': 'Ghost House Pictures', 'id': 768}, ...","[{'iso_3166_1': 'US', 'name': 'United States o...",8/30/12,92.0,...,0,0,0,0,0,0,0,0,0,0
8,[],0,"[{'id': 99, 'name': 'Documentary'}]",en,A chronicle which provides a rare window into ...,1.949044,[],[],1/15/04,84.0,...,0,0,0,0,0,0,0,0,0,0
9,"[{'id': 256377, 'name': 'The Muppet Collection...",0,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...",en,After telling the story of Flint's last journe...,6.902423,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/16/96,100.0,...,1,0,0,0,0,0,0,0,0,0
